# Part1: Popularity Prediction

## Problem 1.5

In [4]:
import os
import os
import json
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.cross_validation import cross_val_predict
from sklearn.ensemble import RandomForestRegressor

filenames = ['tweet_data/tweets_#gohawks.txt', 
             'tweet_data/tweets_#gopatriots.txt',
             'tweet_data/tweets_#nfl.txt',
             'tweet_data/tweets_#patriots.txt',
             'tweet_data/tweets_#sb49.txt',
             'tweet_data/tweets_#superbowl.txt']

# aggregate the training data
with open('tweet_data/output_file.txt', 'w',encoding='utf-8') as outfile:
    for fname in filenames:
        with open(fname,encoding='utf-8') as infile:
            for line in infile:
                outfile.write(line)
# extract features
time_tweet = []
num_retweets = []
num_followers = []
num_url_citations = []
author_names = []

ranking_scores = []
num_hashtags = []

num_replies=[]
num_impressions=[]
num_favorite=[]



with open('tweet_data/output_file.txt',encoding='utf-8') as input_file:
        for line in input_file:
            data = json.loads(line)
            time_tweet.append(data['citation_date'])
            #num_retweets.append(data['metrics']['citations']['total'])
            num_followers.append(data['author']['followers'])

            author_name = data['author']['nick']
            original_author_name = data['original_author']['nick']
            if author_name != original_author_name:
                num_retweets.append(True)
            else:
                num_retweets.append(False)
            num_url_citations.append(len(data['tweet']['entities']['urls']))
            author_names.append(author_name)
           
            ranking_scores.append(data['metrics']['ranking_score'])
            num_hashtags.append(data['title'].count('#'))
            num_replies.append(data['metrics']['citations']['replies'])
            num_impressions.append(data['metrics']['impressions'])
            num_favorite.append(data['tweet']['favorite_count'])
            
            
            

        input_file.close()

# extract features
start_time = 1421222400
hours_passed = int((max(time_tweet)-start_time)/3600)+1 
hourly_num_tweets = [0] * hours_passed
hourly_num_retweets = [0] * hours_passed
hourly_tot_followers = [0] * hours_passed

hourly_time_day = [0] * hours_passed
totnum_tweets = float(len(time_tweet))
hourly_num_url_citations = [0] * hours_passed
hourly_num_authors = [0] * hours_passed
hourly_author_set = [0] * hours_passed
for i in range(0, hours_passed):
    hourly_author_set[i] = set([])

hourly_tot_ranking_scores = [0.0] * hours_passed
hourly_num_hashtags = [0] * hours_passed

hourly_number_of_impressions=[0] * hours_passed
hourly_number_of_replies=[0] * hours_passed
hourly_number_of_favorite_count=[0] * hours_passed


for i in range(int(totnum_tweets)):
    current_hour = int((time_tweet[i]-start_time)/3600)

    hourly_num_tweets[current_hour] += 1
    if num_retweets[i]:
        hourly_num_retweets[current_hour] += 1

    hourly_tot_followers[current_hour] += num_followers[i]

 
    hourly_num_url_citations[current_hour] += num_url_citations[i]
    hourly_author_set[current_hour].add(author_names[i])
   
    hourly_tot_ranking_scores[current_hour] += ranking_scores[i]
    hourly_num_hashtags[current_hour] += num_hashtags[i]
    
    hourly_number_of_impressions[current_hour] +=num_impressions[i]
    hourly_number_of_replies[current_hour] +=num_replis[i]
    hourly_number_of_favorite_count[current_hour] +=num_favorite[i]

        

    for i in range(0, len(hourly_author_set)):
        hourly_num_authors[i] = len(hourly_author_set[i])
    for i in range(len(hourly_time_day)):
        hourly_time_day[i] = i%24
        
# transfer features from one hour to five hour
five_hour_num_tweets = []
five_hour_num_retweets = []
five_hour_tot_followers = []
five_hour_maxnum_followers = []
five_hour_time_day = []
five_hour_num_url_citations = []
five_hour_num_authors = []

five_hour_tot_ranking_scores = []
five_hour_num_hashtags = []

five_hour_num_impressions=[]
five_hour_num_replies=[]
five_hour_num_favorite=[]



for i in range(hours_passed):
    five_hour_num_tweets.append(sum(hourly_num_tweets[i:i+5]))
    five_hour_num_retweets.append(sum(hourly_num_retweets[i:i+5]))
    five_hour_tot_followers.append(sum(hourly_tot_followers[i:i+5]))
    
    five_hour_time_day.append(sum(hourly_time_day[i:i+5]))
    five_hour_num_url_citations.append(sum(hourly_num_url_citations[i:i+5]))
    five_hour_num_authors.append(sum(hourly_num_authors[i:i+5]))
    
    five_hour_tot_ranking_scores.append(sum(hourly_tot_ranking_scores[i:i+5]))
    five_hour_num_hashtags.append(sum(hourly_num_hashtags[i:i+5]))
    
    
    five_hour_num_impressions.append(sum(hourly_number_of_impressions[i:i+5]))
    five_hour_num_replies.append(sum(hourly_number_of_replies[i:i+5]))
    five_hour_num_favorite.append(sum(hourly_number_of_favorite_count[i:i+5]))
    
    
five_hour_num_tweets = five_hour_num_tweets[:-4]
five_hour_num_retweets = five_hour_num_retweets[:-4]
five_hour_tot_followers = five_hour_tot_followers[:-4]

five_hour_time_day = five_hour_time_day[:-4]
five_hour_num_url_citations = five_hour_num_url_citations[:-4]
five_hour_num_authors = five_hour_num_authors[:-4]

five_hour_tot_ranking_scores = five_hour_tot_ranking_scores[:-4]
five_hour_num_hashtags = five_hour_num_hashtags[:-4]


five_hour_num_impressions=five_hour_num_impressions[:-4]
five_hour_num_replies= five_hour_num_replies[:-4]
five_hour_num_favorite=five_hour_num_favorite[:-4]






# Build DataFrame and save to file
target_value = hourly_num_tweets[5:]
target_value.append(0)
data = np.array([five_hour_num_tweets,
                 five_hour_num_retweets,
                 five_hour_tot_followers,
                 five_hour_time_day,
                 five_hour_num_url_citations,
                 five_hour_num_authors,
                 five_hour_tot_ranking_scores,
                 five_hour_num_hashtags,
                 five_hour_num_impressions,
                 five_hour_num_replies,
                 five_hour_num_favorite,
                 target_value])
data = np.transpose(data)
df = DataFrame(data)
df.columns = ['num_tweets',
              'num_retweets',
              'sum_followers',
              'time_of_day', 
              'num_URLs', 
              'num_authors', 
              'ranking_score', 
              'num_hashtags',
              'num_impressions',
              'num_replies',
              'num_favorite',
              'target_value']
if os.path.isdir('./Extracted_data'):
    pass
else:
    os.mkdir('./Extracted_data')
df.to_csv('./Extracted_data/Q5.csv', index = False)   

In [5]:
import os
import os
import json
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.cross_validation import cross_val_predict
from sklearn.ensemble import RandomForestRegressor
#------------------linear regression-----------------------------------------
def average_prediction_error(target_data, cross_predicted_values):
    total_error = 0.0
    for (actual, predicted) in zip(target_data, cross_predicted_values):
        total_error += abs(actual - predicted)
    print(total_error/len(target_data))
    
training_data = pd.read_csv('./Extracted_data/Q5.csv')
target_data = training_data.pop('target_value')
    
training_data1 = training_data[:440] # Before Feb. 1, 8:00 a.m.
training_data2 = training_data[440:452] # Between Feb. 1, 8:00 a.m. and 8:00 p.m.
training_data3 = training_data[452:] # After Feb. 1, 8:00 p.m. 

target_data1 = target_data[:440]
target_data2 = target_data[440:452]
target_data3 = target_data[452:]  

lin_reg = LinearRegression(fit_intercept = False)
cross_predicted_values1 = cross_val_predict(lin_reg, training_data1, target_data1, cv = 10)
cross_predicted_values2 = cross_val_predict(lin_reg, training_data2, target_data2, cv = 10)
cross_predicted_values3 = cross_val_predict(lin_reg, training_data3, target_data3, cv = 10)
print('Average prediction error before Feb. 1, 8:00 a.m.:')
average_prediction_error(target_data1,cross_predicted_values1)

print('Average prediction error between Feb. 1, 8:00 a.m. and 8:00 p.m.:')
average_prediction_error(target_data2,cross_predicted_values2)

print('Average prediction error after Feb. 1, 8:00 p.m.:')
average_prediction_error(target_data3,cross_predicted_values3)

Average prediction error before Feb. 1, 8:00 a.m.:
5470.98762751
Average prediction error between Feb. 1, 8:00 a.m. and 8:00 p.m.:
81748.2161516
Average prediction error after Feb. 1, 8:00 p.m.:
757.530975068


In [6]:
#-------------------------------random forest-------------------------------------------
rf = RandomForestRegressor(n_estimators = 20, max_depth = 9)
cross_predicted_values1 = cross_val_predict(rf, training_data1, target_data1, cv = 10)
cross_predicted_values2 = cross_val_predict(rf, training_data2, target_data2, cv = 10)
cross_predicted_values3 = cross_val_predict(rf, training_data3, target_data3, cv = 10)
print('Average prediction error before Feb. 1, 8:00 a.m.:')
average_prediction_error(target_data1,cross_predicted_values1)

print('Average prediction error between Feb. 1, 8:00 a.m. and 8:00 p.m.:')
average_prediction_error(target_data2,cross_predicted_values2)

print('Average prediction error after Feb. 1, 8:00 p.m.:')
average_prediction_error(target_data3,cross_predicted_values3)

Average prediction error before Feb. 1, 8:00 a.m.:
1922.00670071
Average prediction error between Feb. 1, 8:00 a.m. and 8:00 p.m.:
87542.4708333
Average prediction error after Feb. 1, 8:00 p.m.:
656.249490571


Based on above, we will choose random forest model to predict test data.

In [11]:
filenames = ['test_data/sample1_period1.txt', 
             'test_data/sample2_period2.txt',
             'test_data/sample3_period3.txt',
             'test_data/sample4_period1.txt',
             'test_data/sample5_period1.txt',
             'test_data/sample6_period2.txt',
             'test_data/sample7_period3.txt',
             'test_data/sample8_period1.txt',
             'test_data/sample9_period2.txt',
             'test_data/sample10_period3.txt']

rf = RandomForestRegressor(n_estimators = 20, max_depth = 9)
rf_before = rf.fit(training_data1, target_data1)
rf_during = rf.fit(training_data2, target_data2)
rf_after = rf.fit(training_data3, target_data3)

def predict_tweet(filename):
    time_tweet = []
    num_retweets = []
    num_followers = []
    num_url_citations = []
    author_names = []
    ranking_scores = []
    num_hashtags = []
    
    num_replies=[]
    num_impressions=[]
    num_favorite=[]

    with open('test_data/'+filename,encoding='utf-8') as input_file:
            for line in input_file:
                data = json.loads(line)
                time_tweet.append(data['citation_date'])
                #num_retweets.append(data['metrics']['citations']['total'])
                num_followers.append(data['author']['followers'])

                author_name = data['author']['nick']
                original_author_name = data['original_author']['nick']
                if author_name != original_author_name:
                    num_retweets.append(True)
                else:
                    num_retweets.append(False)
                num_url_citations.append(len(data['tweet']['entities']['urls']))
                author_names.append(author_name)
                ranking_scores.append(data['metrics']['ranking_score'])
                num_hashtags.append(data['title'].count('#'))
                
                num_replies.append(data['metrics']['citations']['replies'])
                num_impressions.append(data['metrics']['impressions'])
                num_favorite.append(data['tweet']['favorite_count'])

                
            

            input_file.close()

    # extract features
    start_time = 1421222400
    hours_passed = int((max(time_tweet)-start_time)/3600)+1 
    hourly_num_tweets = [0] * hours_passed
    hourly_num_retweets = [0] * hours_passed
    hourly_tot_followers = [0] * hours_passed
    hourly_time_day = [0] * hours_passed
    totnum_tweets = float(len(time_tweet))
    hourly_num_url_citations = [0] * hours_passed
    hourly_num_authors = [0] * hours_passed
    hourly_author_set = [0] * hours_passed
    for i in range(0, hours_passed):
        hourly_author_set[i] = set([])
    hourly_tot_ranking_scores = [0.0] * hours_passed
    hourly_num_hashtags = [0] * hours_passed
    
    
    hourly_number_of_impressions=[0] * hours_passed
    hourly_number_of_replies=[0] * hours_passed
    hourly_number_of_favorite_count=[0] * hours_passed




    for i in range(int(totnum_tweets)):
        current_hour = int((time_tweet[i]-start_time)/3600)

        hourly_num_tweets[current_hour] += 1
        if num_retweets[i]:
            hourly_num_retweets[current_hour] += 1

        hourly_tot_followers[current_hour] += num_followers[i]

        hourly_num_url_citations[current_hour] += num_url_citations[i]
        hourly_author_set[current_hour].add(author_names[i])
        
        hourly_tot_ranking_scores[current_hour] += ranking_scores[i]
        hourly_num_hashtags[current_hour] += num_hashtags[i]
        
        
        hourly_number_of_impressions[current_hour] +=num_impressions[i]
        hourly_number_of_replies[current_hour] +=num_replies[i]
        hourly_number_of_favorite_count[current_hour] +=num_favorite[i]


        for i in range(0, len(hourly_author_set)):
            hourly_num_authors[i] = len(hourly_author_set[i])
        for i in range(len(hourly_time_day)):
            hourly_time_day[i] = i%24

    # transfer features from one hour to six hour
    six_hour_num_tweets = []
    six_hour_num_retweets = []
    six_hour_tot_followers = []

    six_hour_time_day = []
    six_hour_num_url_citations = []
    six_hour_num_authors = []
    
    six_hour_tot_ranking_scores = []
    six_hour_num_hashtags = []
    
    six_hour_num_impressions=[]
    six_hour_num_replies=[]
    six_hour_num_favorite=[]
    
    six_hour_num_tweets.append(sum(hourly_num_tweets))
    six_hour_num_retweets.append(sum(hourly_num_retweets))
    six_hour_tot_followers.append(sum(hourly_tot_followers))
    
    six_hour_time_day.append(sum(hourly_time_day))
    six_hour_num_url_citations.append(sum(hourly_num_url_citations))
    six_hour_num_authors.append(sum(hourly_num_authors))
 
    six_hour_tot_ranking_scores.append(sum(hourly_tot_ranking_scores))
    six_hour_num_hashtags.append(sum(hourly_num_hashtags))
    
    six_hour_num_impressions.append(sum(hourly_number_of_impressions))
    
    six_hour_num_replies.append(sum(hourly_number_of_replies))
    six_hour_num_favorite.append(sum(hourly_number_of_favorite_count))
    
#     six_hour_num_tweets = six_hour_num_tweets[:-5]
#     six_hour_num_retweets = six_hour_num_retweets[:-5]
#     six_hour_tot_followers = six_hour_tot_followers[:-5]
#     six_hour_maxnum_followers = six_hour_maxnum_followers[:-5]
#     six_hour_time_day = six_hour_time_day[:-5]
#     six_hour_num_url_citations = six_hour_num_url_citations[:-5]
#     six_hour_num_authors = six_hour_num_authors[:-5]
#     six_hour_num_mentions = six_hour_num_mentions[:-5]
#     six_hour_tot_ranking_scores = six_hour_tot_ranking_scores[:-5]
#     six_hour_num_hashtags = six_hour_num_hashtags[:-5]

    # Build DataFrame and save to file
#     target_value = hourly_num_tweets[6:]
#     target_value.append(0)
    data = np.array([six_hour_num_tweets,
                     six_hour_num_retweets,
                     six_hour_tot_followers,
                     six_hour_time_day,
                     six_hour_num_url_citations,
                     six_hour_num_authors,
                     six_hour_tot_ranking_scores,
                     six_hour_num_hashtags,
                     six_hour_num_impressions,
                     six_hour_num_replies,
                     six_hour_num_favorite
                     ])
    data = np.transpose(data)
    df = DataFrame(data)
    df.columns = ['num_tweets',
                  'num_retweets',
                  'sum_followers',
                  'time_of_day', 
                  'num_URLs', 
                  'num_authors',
                  'ranking_score', 
                  'num_hashtags', 
                  'num_impressions',
                  'num_replies',
                  'num_favorite'
                  ]
    if os.path.isdir('./Extracted_data'):
        pass
    else:
        os.mkdir('./Extracted_data')
    df.to_csv('./Extracted_data/Q5_'+filename[:-4]+'.csv', index = False)
    
    testing_data = pd.read_csv('./Extracted_data/Q5_'+filename[:-4]+'.csv')
#     target_data = testing_data.pop('target_value')
    predicted_y = []
    if filename[-5] == '1':
        predicted_y = rf_before.predict(testing_data)
        print(predicted_y)
    elif filename[-5] == '2':
        predicted_y = rf_during.predict(testing_data)
        print(predicted_y)
    else:
        predicted_y = rf_after.predict(testing_data)
        print(predicted_y)
    

    

In [12]:
predict_tweet('sample1_period1.txt')
predict_tweet('sample2_period2.txt')
predict_tweet('sample3_period3.txt')
predict_tweet('sample4_period1.txt')
predict_tweet('sample5_period1.txt')
predict_tweet('sample6_period2.txt')
predict_tweet('sample7_period3.txt')
predict_tweet('sample8_period1.txt')
predict_tweet('sample9_period2.txt')
predict_tweet('sample10_period3.txt')
             

[ 475.95]
[ 4485.55]
[ 539.125]
[ 472.7]
[ 493.55]
[ 4728.4]
[ 36.8]
[ 66.95]
[ 1689.965]
[ 53.65]
